![alt text](images/uspas.png)
# VUV and X-ray Free Electron Lasers
# Lab Day 3: 1D FEL Simulations with ZFEL

##### Instructors: D. Nguyen, P. Anisimov, N. Neveu
##### Teaching Assistant: Y.S. Li
----------

# 1. Import functions

In [ ]:
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.style.use(r'PaperDoubleFig.mplstyle')

from zfel import sase1d_input_part

# 2. Select zfel inputs and outputs   

    '''
    SASE 1D FEL run function, INPUTS and OUTPUTS
    Input:
    npart                       # n-macro-particles per bucket 
    s_steps                     # n-sample points along bunch length
    z_steps                     # n-sample points along undulator
    energy                      # electron energy [eV]
    eSpread                     # relative rms energy spread [ ]
    emitN                       # normalized transverse emittance [mm-mrad]
    currentMax                  # peak current [Ampere]
    beta                        # mean beta [meter]
    unduPeriod                  # undulator period [meter]
    unduK                       # undulator parameter, K [ ]
    unduL                       # length of undulator [meter]
    radWavelength               # seed wavelength [meter] (if you had a seed laser), used only in single-freuqency runs
    iopt                        # 'sase' or 'seeded'
    P0                          # small seed input power [W]
    constseed                   # whether we want to use constant  random seed for reproducibility, 1 Yes, 0 No
    particle_position           # particle information with positions in meter and eta
    hist_rule                   # different rules to select number of intervals to generate the histogram of eta value in a bucket

    Output:
    z                           # longitudinal steps along undulator
    power_z                     # power profile along undulator                    
    s                           # longitudinal steps along beam
    power_s                     # power profile along beam    
    rho                         # FEL Pierce parameter
    gainLength                  # 1D FEL gain Length
    resWavelength               # resonant wavelength
    thet                        # output phase
    eta                         # output energy in unit of mc2
    spectrum                    # spectrum power
    freq                        # frequency in ev
    Ns                          # real number of examples
    '''

# Electron beam description
These paramters based on LCLS hard x-ray.  
LCLS parameters are posted periodically: https://lcls.slac.stanford.edu/sites/lcls.slac.stanford.edu/files/LCLS-Parameters-11-15-19.pdf

In [ ]:
currentMax = 3e3  # maximum current [A]
energy     = 13.6e9  # electron energy [eV]
eSpread    = 1.e-4  # relative rms energy spread sigma_E/E
emitN      = 0.4e-6  # normalized emittance [m-rad]
beta       = 18  # mean beta function of the beam [m]

# Undulator description

In [ ]:
unduPeriod = 3.e-2  # undulator period []
unduL      = 2500*unduPeriod  # undulator length [m]
z_steps    = 100  # n-sample points along undulator; it also defines time step along bunch
unduK      = np.full(z_steps, np.sqrt(2)*2.475)  # undulator parametr
dEdz       = None  # not implemented

# X-ray radiation description

In [ ]:
iopt = 'sase'  # The only mode of operation
P0   = 0.e4 * 1e9  # small seed input power [W]
                # corrected by 1e-9 error in the code
radWavelength =  1.5088e-10 # resonant wavelength
s_steps = 10000 # n-samples along bunch; tail is at 0; it also defines bunch length

# Technical description

In [ ]:
Nruns = None   # not implemented
npart = 2e9    # n-macro-particles per bucket
constseed = 1  # use constant random seed for reproducibility,
               # 1 Yes, 0 No
particle_position = None  # particle distro with positions in
                          # meter and eta
hist_rule  = 'square-root'  # 'square-root' or 'sturges' or
                            # 'rice-rule' or 'self-design',
                            # number of intervals to generate
                            # the histogram of eta value in a bucket

# 3. Run zfel
This may take a while...

In [ ]:
# Load input parameters into zfel 
inp_struct={'Nruns':Nruns, 'npart':npart,'s_steps':s_steps,'z_steps':z_steps,\
            'energy':energy,'eSpread':eSpread, 'dEdz':dEdz,\
            'emitN':emitN,'currentMax':currentMax,'beta':beta,\
            'unduPeriod':unduPeriod,'unduK':unduK,'unduL':unduL, \
            'radWavelength':radWavelength,'iopt':iopt,'P0':P0,\
            'constseed':constseed,'particle_position':particle_position,'hist_rule':hist_rule}

z,power_z,s,power_s,rho,detune,field,\
field_s,gainLength,resWavelength,\
thet_out,eta_out,bunching,spectrum,freq,Ns,history = sase1d_input_part.sase(inp_struct)

# 4. Plot results
NOTE: some data has a zero in the first position, avoid it by indexing sarting from 1 like this: data[1:]

In [ ]:
plt.semilogy(z, 1e-9*power_z)
plt.xlabel("Undulator distance, m")
plt.ylabel("Power (log scale), GW")
plt.xlim([0, unduL])
plt.twinx()
plt.plot(z, 1e-9*power_z, 'r')
plt.ylabel("Power (lin scale), GW")
plt.ylim([0, 50])
plt.show()

plt.semilogy(1e-3*freq, 1e12*spectrum[1:,-1])
plt.xlabel("Photon energy, keV")
plt.ylabel("Power spectrum (log scale), a.u.")
# plt.ylim([1e-7, 15])
plt.twinx()
plt.plot(1e-3*freq, 1e12*spectrum[1:,-1], 'r')
# plt.ylim([0, 5.])
plt.ylabel("Power spectrum (lin scale), a.u.")
plt.show()

In [ ]:
power_s = output['power_s']
plt.figure()
for i in range(power_s.shape[0]):
    plt.plot(s,power_s[i,:])
plt.title('Power along the beam')
plt.xlabel('s (um)')
plt.ylabel('power at different z positions (W)')
plt.show()

plt.plot(1e6*s, 1e-9*power_s[-1], '-o')
plt.xlabel(r"Beam coordinate, $\mu m$")
plt.ylabel("Power, GW")

# 5. Calculate gain length using Pierce parameter
See page and 57 in book and lecture notes. How does this compare to the zfel rho calculation?

In [ ]:
#zfel rho
print(rho[0])

# 6. Calculate gain length using power curve from zfel results
See page 59 in the book.

# 7. Repeat #5-6 with a new machine. 
Pick another set of parameters, can be LCLS or Marie in a different regime or a different machine.  
Write 1-2 sentences commenting on the differences. 
Try to pick a machine you would consider using for your FEL project.   
You can work with your project team.
Leave your completed notebook in your student directory.